# Begginer level approach

## Imports

In [1]:
import pandas as pd
import tensorflow as tf
import keras_nlp
from sklearn.model_selection import train_test_split

E0000 00:00:1726615927.845321    5211 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:479
D0917 23:32:07.853781579    5211 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D0917 23:32:07.853795412    5211 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D0917 23:32:07.853798695    5211 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D0917 23:32:07.853801100    5211 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D0917 23:32:07.853803427    5211 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defa

In [2]:
try:
    # detect and init the TPU
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)
    print("All devices: ", tf.config.list_logical_devices('TPU'))
except ValueError:
    strategy = tf.distribute.get_strategy()  # default strategy if no TPU available

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1726615933.935542    5211 service.cc:145] XLA service 0x57c0db737490 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1726615933.935597    5211 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1726615933.935602    5211 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1726615933.935605    5211 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1726615933.935608    5211 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1726615933.935610    5211 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1726615933.935613    5211 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1726615933.935615    5211 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1726615933.935618    5211 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

In [3]:
training_data = pd.read_csv('/kaggle/input/contradictory-my-dear-watson/train.csv')

df_train, df_test = train_test_split(training_data, test_size=0.2, random_state=42)

## Model

In [5]:
# Asegúrate de que 'premise' y 'hypothesis' sean listas de strings
premises = df_train['premise'].astype(str).tolist()
hypotheses = df_train['hypothesis'].astype(str).tolist()

# Define las labels (etiquetas) para entrenar
labels = tf.convert_to_tensor(df_train['label'])

# Verifica que todas las entradas tengan la misma longitud
assert len(premises) == len(hypotheses) == len(labels), "Las dimensiones de las entradas no coinciden."

# Crea un dataset a partir de las entradas
train_dataset = tf.data.Dataset.from_tensor_slices(((premises, hypotheses), labels)).batch(32)

# Define el modelo dentro del scope de la estrategia
with strategy.scope():
    classifier = keras_nlp.models.DistilBertClassifier.from_preset(
        "distil_bert_base_multi",
        num_classes=3
    )
    classifier.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )

# Entrena el modelo con el dataset
classifier.fit(train_dataset, epochs=10)


Epoch 1/10


2024-09-17 23:34:08.258445: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node StatefulPartitionedCall.
I0000 00:00:1726616049.583813    6013 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(9aa6c084ffb6f62a:0:0), session_name()
I0000 00:00:1726616101.944723    6013 tpu_compile_op_common.cc:245] Compilation of 9aa6c084ffb6f62a:0:0 with session name  took 52.360850104s and succeeded
I0000 00:00:1726616102.121964    6013 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(9aa6c084ffb6f62a:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_iterator_8776224269759706568", property.function_library_fingerprint = 17777763372862248066, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z 

303/303 ━━━━━━━━━━━━━━━━━━━━ 88s 83ms/step - accuracy: 0.3273 - loss: 1.1129
Epoch 2/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 26s 83ms/step - accuracy: 0.3205 - loss: 1.1023
Epoch 3/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 27s 84ms/step - accuracy: 0.3483 - loss: 1.0982
Epoch 4/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 27s 84ms/step - accuracy: 0.3375 - loss: 1.0976
Epoch 5/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 27s 85ms/step - accuracy: 0.3653 - loss: 1.0969
Epoch 6/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 27s 84ms/step - accuracy: 0.3530 - loss: 1.0977
Epoch 7/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 26s 83ms/step - accuracy: 0.3778 - loss: 1.0976
Epoch 8/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 27s 83ms/step - accuracy: 0.3537 - loss: 1.0992
Epoch 9/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 27s 84ms/step - accuracy: 0.3485 - loss: 1.1032
Epoch 10/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 27s 84ms/step - accuracy: 0.3546 - loss: 1.0977


## Evaluation on test split

In [6]:
premises = df_test['premise'].astype(str).tolist()
hypotheses = df_test['hypothesis'].astype(str).tolist()

# Define las labels (etiquetas) para entrenar
labels = tf.convert_to_tensor(df_test['label'])

test_dataset = tf.data.Dataset.from_tensor_slices(((premises, hypotheses), labels)).batch(32)

test_loss, test_accuracy = classifier.evaluate(test_dataset)
print(f'Precisión en conjunto de test: {test_accuracy:.4f}')

2024-09-17 23:39:44.501787: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.
I0000 00:00:1726616384.868038    6002 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(fb80fa8d18cef694:0:0), session_name()


 7/76 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.4070 - loss: 1.0992

I0000 00:00:1726616386.984676    6002 tpu_compile_op_common.cc:245] Compilation of fb80fa8d18cef694:0:0 with session name  took 2.11658979s and succeeded
I0000 00:00:1726616386.997314    6002 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(fb80fa8d18cef694:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_iterator_895186845689594259", property.function_library_fingerprint = 5105639495897639377, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "4,512,;4,512,;4,;", property.guaranteed_constants_size = 0, embedding_partitions_fingerprint = "1688352644216761960")
I0000 00:00:1726616386.997341    6002 tpu_compilation_cache_interface.cc:541] After adding entry for key fb80f

73/76 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.3447 - loss: 1.0979

I0000 00:00:1726616388.476330    5992 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(378a566afe570aab:0:0), session_name()


76/76 ━━━━━━━━━━━━━━━━━━━━ 9s 54ms/step - accuracy: 0.3441 - loss: 1.0978
Precisión en conjunto de test: 0.3366


I0000 00:00:1726616391.007210    5992 tpu_compile_op_common.cc:245] Compilation of 378a566afe570aab:0:0 with session name  took 2.530840559s and succeeded
I0000 00:00:1726616391.024011    5992 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(378a566afe570aab:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_one_step_on_iterator_895186845689594259", property.function_library_fingerprint = 5105639495897639377, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "3,512,;3,512,;3,;", property.guaranteed_constants_size = 0, embedding_partitions_fingerprint = "1688352644216761960")
I0000 00:00:1726616391.024045    5992 tpu_compilation_cache_interface.cc:541] After adding entry for key 378a